# Modeling Test #2
### 사용자 입력값 + VOD 줄거리

<br><br><hr>

## 00. 기본 설정

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 한글 font 설정
import platform
import matplotlib.font_manager as fm

#matplotlib 패키지 한글 깨짐 처리 시작
#------------------------------------------------------------------------------------
# 운영체제별 한글 폰트 설정

if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
    
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [3]:
# 글씨 선명하게 출력하는 설정

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

<br><br><hr>

## 01. 데이터 불러오기

In [4]:
movies = pd.read_csv('..\data\movies_4000_tmdb_genre.csv')
print(movies.shape)   # (4241, 6)
movies.head(3)

(4241, 6)


,asset_nm_전처리,ct_cl,genre_of_ct_cl,summary_최신순,최신순,genre_tmdb
0,귀멸의 칼날: 남매의 연,영화,애니메이션,혈귀의 습격으로 가족을 잃은 소년 ‘탄지로’. 유일하게 살아남은 여동생 ‘네즈코’마...,2019-03-29,"Animation, Action, Fantasy, Thriller"
1,색에 놀다,영화,에로틱,하얀 색의 순수하고 착한 사랑을 꿈꾸는 25살 모태 솔로 지수. 그녀의 짝사랑 상대...,2017-01-01,"Thriller, Drama, Romance"
2,돌이킬 수 없는 주말,영화,공포/스릴러,베키는 결혼을 앞두고 친구 수잔과 함께 다트무어로 여행을 떠난다. 그곳에서 신비한 ...,2015-09-18,"Drama, Horror, Mystery"


In [5]:
movies['genre_of_ct_cl'].value_counts()

genre_of_ct_cl
드라마        1011
액션/어드벤쳐     977
공포/스릴러      639
성인          340
멜로          329
코미디         265
애니메이션       169
SF/환타지      159
다큐멘터리       126
기타           88
무협           37
로맨틱코미디       37
에로틱          35
단편           12
서부            7
뮤지컬           6
역사            2
인물            2
Name: count, dtype: int64

##### *>> 장르가 '성인' 또는 '기타'인 행와 장르의 value_counts가 40개 미만인 행은 삭제*

In [6]:
# 각 장르의 수 를 genre_counts 변수에 저장
genre_counts = movies['genre_of_ct_cl'].value_counts()

In [7]:
# 삭제할 조건 생성
delete_conditions = (movies['genre_of_ct_cl'] == '성인') | (movies['genre_of_ct_cl'] == '기타') | (movies['genre_of_ct_cl'].isin(genre_counts[genre_counts < 40].index))
filtered_movies = movies[~delete_conditions]

In [8]:
# 결과 확인
print(filtered_movies.shape)
filtered_movies['genre_of_ct_cl'].value_counts()

### (4241, 6) >> (3675, 6) >> 566개의 행 삭제

(3675, 6)


genre_of_ct_cl
드라마        1011
액션/어드벤쳐     977
공포/스릴러      639
멜로          329
코미디         265
애니메이션       169
SF/환타지      159
다큐멘터리       126
Name: count, dtype: int64

##### *>> 사용하지 않을 열 drop*

In [9]:
col_to_drop = ['ct_cl', '최신순', 'genre_tmdb']
drop_movies = filtered_movies.drop(columns=col_to_drop, axis=1)

drop_movies.head(3)

,asset_nm_전처리,genre_of_ct_cl,summary_최신순
0,귀멸의 칼날: 남매의 연,애니메이션,혈귀의 습격으로 가족을 잃은 소년 ‘탄지로’. 유일하게 살아남은 여동생 ‘네즈코’마...
2,돌이킬 수 없는 주말,공포/스릴러,베키는 결혼을 앞두고 친구 수잔과 함께 다트무어로 여행을 떠난다. 그곳에서 신비한 ...
3,섹스 앤 머니,액션/어드벤쳐,갱단 두목 페페는 라이벌 갱단 두목 조조와 세력 다툼을 벌이다 쫓기는 신세가 된다....


<br><br><hr>

## 02. 한글 텍스트 전처리

In [10]:
from konlpy.tag import Okt
import re

In [11]:
# 줄거리 null값 확인
drop_movies['summary_최신순'].isnull().sum()

np.int64(0)

In [12]:
# 정규표현식을 이용해 숫자를 공백으로 변경 (정규 표현식으로 \d는 숫자 의미)
drop_movies['summary_최신순'] = drop_movies['summary_최신순'].apply(lambda x : re.sub(r"\d+", "", x))

In [13]:
# 정규표현식을 이용해 특수문자를 공백으로 변경
drop_movies['summary_최신순'] = drop_movies['summary_최신순'].apply(lambda x: re.sub(r"[^가-힣a-zA-Z0-9\s]", "", x))

In [14]:
# Okt 객체 생성
okt = Okt()

In [15]:
def okt_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 변환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV

In [17]:
# okt 객체의 morphs() 객체를 이용한 tokenizer 사용, ngram_rage는 (1, 2)
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer,
                             ngram_range=(1, 2),  # unigram + bigram
                             min_df=2,
                             max_df=0.85)
tfidf_vect.fit(drop_movies['summary_최신순'])
tfidf_matrix = tfidf_vect.transform(drop_movies['summary_최신순'])

<br><br><hr>

## 03. 토큰화된 문서를 장르 기준으로 LDA 수행

In [20]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# stopwords 가져오기
with open('..\data\stopwords\combined_stopwords.txt', 'r', encoding='utf-8') as stopwords_file:
    lda_stopwords = stopwords_file.read().splitlines()

In [22]:
drop_movies['genre_of_ct_cl'].value_counts()

genre_of_ct_cl
드라마        1011
액션/어드벤쳐     977
공포/스릴러      639
멜로          329
코미디         265
애니메이션       169
SF/환타지      159
다큐멘터리       126
Name: count, dtype: int64

In [23]:
# 1. 장르별 데이터 분리
genres = ['드라마', '액션/어드벤쳐', '공포/스릴러', '애니메이션', '코미디', '멜로','SF/환타지', '다큐멘터리']

# 장르별 데이터프레임 생성
genre_dfs = {genre: drop_movies[drop_movies['genre_of_ct_cl'] == genre] for genre in genres}

In [24]:
# 불용어 리스트에 해지, 시청, 가능, 서비스 추가
additional_stop_words = ['해지', '시청', '가능', '서비스', '가능합니다', '평생', '소장',
                         'of', 'the', 'a', 'in', 'is', 'her', 'and', 'to', 'it', 'he', 'she', 'his',
                         'on', 'who', 'with']
lda_stopwords.extend(additional_stop_words)

In [25]:
lda_results = {}

for genre, drop_movies in genre_dfs.items():
    # 문서 개수 확인
    doc_count = len(drop_movies['summary_최신순'])
    print(f"{genre} 문서 개수: {doc_count}")
    # 2. 토픽 모델링을 위한 전처리
    # TF-IDF로 변환된 데이터를 장르별로 LDA에 입력할 수 있는 형태로 준비
    
    # LDA는 Bag-of-Words(BoW) 지원하므로 CountVectorizer로 BoW를 생성
    vectorizer = CountVectorizer(stop_words=lda_stopwords,
                                 tokenizer=okt_tokenizer,
                                 max_features=5000,
                                 max_df=0.80,
                                 min_df=2,
                                 # ngram_range=(1, 2)
                                )
    bow_matrix = vectorizer.fit_transform(drop_movies['summary_최신순'])
    
    # 3. LDA 모델 생성 및 학습
    # LatentDirichletAllocation을 사용하여 장르별로 토픽을 추출
    # 8개의 토픽 추출
    lda = LatentDirichletAllocation(n_components=5, random_state=42)
    lda.fit(bow_matrix)
    
    # 4. 토픽별 주요 단어 추출
    feature_names = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]  # 상위 10개 단어
        topics[f"Topic {topic_idx + 1}"] = top_words
    
    lda_results[genre] = topics

드라마 문서 개수: 1011
액션/어드벤쳐 문서 개수: 977
공포/스릴러 문서 개수: 639
애니메이션 문서 개수: 169
코미디 문서 개수: 265
멜로 문서 개수: 329
SF/환타지 문서 개수: 159
다큐멘터리 문서 개수: 126


In [26]:
# 5. 결과 출력
for genre, topics in lda_results.items():
    print(f"--- {genre} 장르 ---")
    for topic, words in topics.items():
        print(f"{topic}: {', '.join(words)}")
    print("\n")

--- 드라마 장르 ---
Topic 1: 사람, 시작, 적, 남편, 친구, 가족, 사랑, 아들, 남자, 다시
Topic 2: 시작, 마을, 적, 아버지, 사람, 집, 사랑, 친구, 엄마, 마음
Topic 3: 적, 사랑, 시작, 사람, 삶, 딸, 집, 남자, 엄마, 여자
Topic 4: 시작, 적, 소녀, 사건, 사람, 친구, 학교, 죽음, 마음, 곳
Topic 5: 시작, 사랑, 적, 사건, 사람, 이야기, 아들, 영화, 집, 꿈


--- 액션/어드벤쳐 장르 ---
Topic 1: 경찰, 시작, 조직, 적, 마약, 범죄, 사건, 팀, 가족, 목숨
Topic 2: 시작, 적, 사람, 세계, 테러, 조직, 딸, 납치, 작전, 친구
Topic 3: 시작, 적, 사람, 세상, 명, 곳, 가문, 황제, 요, 미국
Topic 4: 사건, 시작, 적, 부대, 임무, 딸, 작전, 살인, 조직, 목숨
Topic 5: 시작, 사건, 지구, 사람, 곳, 힘, 중국, 비밀, 적, 천


--- 공포/스릴러 장르 ---
Topic 1: 집, 사람, 시작, 곳, 마을, 적, 친구, 비밀, 가족, 사건
Topic 2: 사건, 적, 집, 살인, 시작, 사람, 친구, 죽음, 발견, 마을
Topic 3: 시작, 사건, 적, 집, 사람, 딸, 친구, 곳, 남편, 아들
Topic 4: 잭, 친구, 사건, 사람, 대니, 시작, 집, 남자, 적, 병원
Topic 5: 시작, 적, 사건, 살인, 발견, 가족, 남자, 엄마, 사람, 집


--- 애니메이션 장르 ---
Topic 1: 친구, 세상, 모험, 도시, 슈퍼, 있을까, 납치, 빼꼼, 비밀, 꽁꽁
Topic 2: 친구, 세계, 소녀, 마을, 사람, 모험, 곳, 집, 세상, 향
Topic 3: 세상, 친구, 왕자, 숲, 있을까, 마법, 시작, 드래곤, 인간, 마을
Topic 4: 시작, 사람, 사건, 생활, 꿈, 세계, 북극, 다시, 있을까, 적
Topic 5: 시작, 친구, 가족, 위기, 소년, 적, 비밀, 소녀, 꿈, 문


--- 코미

In [160]:
# 개별 토픽별로 각 word 피처가 얼마나 많이 그 토픽에 할당되었는지에 대한 수치를 가지고 있음
# 높을 값일수록 해당 word 피쳐는 그 토픽의 중심 word가 됨
# print(lda.components_.shape)
# lda.components_

<hr>

## 04. 토픽별로 연관도가 높은 순으로 word 나열

In [27]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic #", topic_index)
        
        # components_ array에서 가장 값이 큰 순으로 정렬했을 때 그 값의 array 인덱스 반환
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]
        
        # top_indexes 대상인 인덱스별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)

In [28]:
# CountVectorizer 객체 내의 전체 word 명칭을 get_features_names_out()를 통해 추출
feature_names = vectorizer.get_feature_names_out()

In [29]:
# 토픽별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

##### 각 토픽(topic)이 해당 텍스트 집합에서 나타내는 잠재적인 주제를 나타냄
##### 각 토픽에 포함된 단어는 해당 주제를 구성하는 주요 단어들로, 이를 기반하여 토픽을 해석할 수 있음

Topic # 0
적 시작 말 후보 팬 했다 개 하지 선수 스타 북 세계 했던 어머니 무대
Topic # 1
적 사람 씨 향 한국 네팔 현 역사 사이 여왕 도시 세상 비 충격 교육
Topic # 2
영화 적 사람 가장 이야기 세계 공연 삶 대한 시작 주년 더 교황 과학자 한국
Topic # 3
적 시작 이자 마마 마지막 새 무 세계 담은 사랑 이야기 문제 도시 무대 에는
Topic # 4
삶 이야기 사람 대한 인생 영화 통해 사랑 세상 시작 적 다큐멘터리 가족 가장 했다


<hr>

## 05. 추천에 적용하기

1. LDA는 각 콘텐츠(영화 줄거리)에 대해 토픽 분포 계산
2. 사용자의 선호 토픽 결정
    - 사용자의 입력(예: "액션 영화를 추천해줘")을 바탕으로 해당 토픽을 선택
    - 또는 사용자가 이전에 시청한 콘텐츠의 토픽 분포를 활용하여 선호도를 추론
3. 추천 콘텐츠 선정
    - LDA 결과에서 사용자의 선호 토픽과 가장 유사한 콘텐츠를 계산하여 추천
    - 코사인 유사도 사용
4. 결과 반환
    - 유사도가 높은 콘텐츠(영화) 상위 5개를 추천

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
import time

In [31]:
def recommend_topic_with_movies(user_input, vectorizer, lda_model, feature_names, movies, no_top_words=10):
    # 사용자 입력을 바탕으로 가장 유사한 토픽을 추천
    # 1. 사용자 입력 텍스트를 BoW(Bag of Words)로 변환
    user_input_bow = vectorizer.transform([user_input])
    
    # 2. LDA 모델을 통해 사용자 입력의 토픽 분포 추출
    user_topic_distribution = lda_model.transform(user_input_bow)
    
    # 3. 가장 높은 확률을 가진 토픽 인덱스 선택
    recommended_topic_idx = np.argmax(user_topic_distribution)
    
    # 4. 해당 토픽의 주요 단어 추출
    topic_terms = lda_model.components_[recommended_topic_idx]
    top_word_indices = topic_terms.argsort()[:-no_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_word_indices]
    
    print(f"추천 토픽: Topic #{recommended_topic_idx + 1}")
    print(f"주요 단어: {', '.join(top_words)}")
    
    # 5. 영화 데이터에서 해당 토픽의 영화 선택
    topic_distributions = lda_model.transform(vectorizer.transform(movies['summary_최신순']))  # 영화별 토픽 분포
    topic_movies = movies[np.argmax(topic_distributions, axis=1) == recommended_topic_idx]
    
    # 랜덤으로 3개 추출
    recommended_movies = topic_movies.sample(n=min(3, len(topic_movies)))
    print("\n추천 영화:")
    print(recommended_movies[['asset_nm_전처리']])
    
    return recommended_topic_idx, top_words, recommended_movies

In [37]:
recommended_topic_idx, top_words, recommended_movies = recommend_topic_with_movies(
    user_input='가장 이야기 세계 공연',
    vectorizer=vectorizer,
    lda_model=lda,
    feature_names=feature_names,
    movies=drop_movies
)

추천 토픽: Topic #3
주요 단어: 영화, 적, 사람, 가장, 이야기, 세계, 공연, 삶, 대한, 시작

추천 영화:
         asset_nm_전처리
2830  나는 마을 방과후 교사입니다
863             머슴 바울
311        의혹을 파는 사람들


In [36]:
# LDA 결과로 콘텐츠-토픽 분포 추출
topic_distributions = lda.transform(bow_matrix) # bow_matrix는 CountVectorizer로 변환된 콘텐츠 데이터
topic_distributions[:10]

array([[0.00815096, 0.96746229, 0.00815803, 0.00810138, 0.00812733],
       [0.00961836, 0.00960672, 0.96143457, 0.00967035, 0.00966999],
       [0.0255923 , 0.02520889, 0.89629285, 0.02609128, 0.02681468],
       [0.01678379, 0.01681218, 0.93245745, 0.01687777, 0.01706882],
       [0.01056484, 0.01056791, 0.01063257, 0.01061219, 0.95762249],
       [0.24519654, 0.00381366, 0.7432946 , 0.00383023, 0.00386498],
       [0.01367882, 0.01383822, 0.01350489, 0.01353063, 0.94544744],
       [0.01123035, 0.01119012, 0.01125864, 0.95513882, 0.01118207],
       [0.73981999, 0.24601115, 0.00474322, 0.00470261, 0.00472303],
       [0.98238722, 0.00439152, 0.00441163, 0.00441428, 0.00439535]])